In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import numpy as np
import tensorflow as tf
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plt
import os

In [8]:
path_train="../input/skin-cancer/train"
path_test="../input/skin-cancer/test"

In [9]:
train_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=path_train,target_size=(224,224),batch_size=10)
test_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=path_test,target_size=(224,224),batch_size=10)


In [10]:
vgg16_model=VGG16()

In [11]:
model=keras.Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

In [12]:
for layer in model.layers:
    layer.trainable=False

In [13]:
model.add(keras.layers.Dense(2,activation='softmax'))

In [14]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [15]:
history =model.fit_generator(train_batches,epochs=30,verbose=1,shuffle=True)

In [34]:
eva=model.evaluate(test_batches,verbose=1)

In [35]:
epoch_count = range(1,len(history.history['loss'])+1)
plt.figure(figsize=(10,15))
plt.plot(history.history['loss'])
plt.xticks(epoch_count)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
"""""
x_train=[]
y_train=[]
x_test=[]
y_test=[]

category={'benign':0,'malignant':1}
""""

**

In [ ]:
"""""
for i in sorted(os.listdir(path_train)):

    path=os.path.join(path_train,i)
    for j in os.listdir(path):
        path_img=os.path.join(path,j)
        image=load_img(path_img,target_size=(64,64))
        image = tf.image.rgb_to_grayscale(image)
        image=img_to_array(image)
        image=image.reshape(-1,64,64,1)
        image=image/255.0
        x_train.append(image)
        y_train.append(category[i])
x_train=np.array(x_train)
y_train=np.array(y_train)
""""

In [2]:
batch_size = 32
image_w = 64
image_h = 64
seed = 123
data_dir = '../input/skin-cancer/train'

train_df = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  label_mode='categorical',
  validation_split=0.2,
  subset="training",
  seed=seed,
  image_size=(image_w,image_h),
  batch_size=batch_size,
  color_mode='grayscale')

data_dir_test='../input/skin-cancer/test'

val_df = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_test,
 label_mode='categorical',
  validation_split=0.2,
  subset="validation",
  seed=seed,
  image_size=(image_w,image_h),
  batch_size=batch_size,
  color_mode='grayscale')

In [3]:
cnn=Sequential()

In [4]:
cnn.add(Conv2D(265, kernel_size=3, activation='relu',input_shape=(64,64,1),padding="valid"))
cnn.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid"))
cnn.add(BatchNormalization())
cnn.add(Dropout(0.2))

cnn.add(Conv2D(128, kernel_size=3, activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid"))
cnn.add(BatchNormalization())
cnn.add(Dropout(0.2))

cnn.add(Conv2D(64, kernel_size=3, activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid"))
cnn.add(BatchNormalization())
cnn.add(Dropout(0.2))

cnn.add(Conv2D(32, kernel_size=3, activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid"))
cnn.add(BatchNormalization())
cnn.add(Dropout(0.2))

cnn.add(Flatten())
cnn.add(Dense(2, activation = "softmax"))

In [5]:
cnn.compile(optimizer = 'Adam' , loss = "categorical_crossentropy", metrics=["accuracy"])

In [16]:
cnn_history=cnn.fit(train_df,batch_size = 100,epochs =30,shuffle=True)

In [33]:
cnn_eva=cnn.evaluate(val_df)

In [27]:
epoch_count = range(1,len(cnn_history.history['loss'])+1)
plt.figure(figsize=(10,15))
plt.plot(cnn_history.history['loss'])
plt.xticks(epoch_count)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

In [31]:
epoch_count = range(1,len(cnn_history.history['loss'])+1)
plt.figure(figsize=(10,15))
plt.plot(cnn_history.history['loss'])
plt.plot(history.history['loss'])
plt.legend(['cnn ','VGG16'])
plt.xticks(epoch_count)
plt.title('VGG16 model loss Vs CNN ')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()